<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/337_SalesEnab_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Summary Generation Utilities

In [ ]:
"""LLM Summary Generation Utilities

Generate concise executive summaries using LLM.
"""

from typing import Dict, Any, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from config import SalesEnablementOrchestratorConfig


def generate_executive_summary(
    state: Dict[str, Any],
    config: SalesEnablementOrchestratorConfig
) -> Optional[str]:
    """
    Generate an LLM-powered executive summary of the sales enablement report.

    Args:
        state: Complete orchestrator state with all insights
        config: Configuration with LLM settings

    Returns:
        Executive summary string, or None if LLM fails
    """
    if not config.enable_llm_summary:
        return None

    try:
        # Extract key metrics from state
        pipeline_summary = state.get("pipeline_summary", {})
        rep_performance = state.get("rep_performance_summary", [])
        top_leads = state.get("top_priority_leads", [])
        rep_nudges = state.get("rep_nudges", [])
        stalled_deals = state.get("stalled_deals", [])
        at_risk_deals = state.get("at_risk_deals", [])
        win_patterns = state.get("win_patterns", [])
        loss_patterns = state.get("loss_patterns", [])

        # Build context for LLM
        context = f"""Sales Enablement Report Data:

Pipeline Health:
- Total Deals: {pipeline_summary.get('total_deals', 0)}
- Active Deals: {pipeline_summary.get('active_deals', 0)}
- Pipeline Value: ${pipeline_summary.get('total_pipeline_value', 0):,.0f}
- Weighted Pipeline Value: ${pipeline_summary.get('weighted_pipeline_value', 0):,.0f}
- Win Rate: {pipeline_summary.get('win_rate', 0)*100:.1f}%
- Stalled Deals: {pipeline_summary.get('stalled_deals_count', 0)}
- At-Risk Deals: {pipeline_summary.get('at_risk_deals_count', 0)}

Rep Performance:
"""
        for rep in rep_performance[:3]:  # Top 3 reps
            context += f"- {rep.get('rep_name', 'Unknown')}: {rep.get('active_deals', 0)} deals, ${rep.get('pipeline_value', 0):,.0f} pipeline, {rep.get('quota_achievement', 0)*100:.1f}% quota"
            if rep.get('needs_coaching'):
                context += " (⚠️ Needs Coaching)"
            context += "\n"

        context += f"""
Top Priority Leads: {len(top_leads)} leads identified
Rep Nudges: {len(rep_nudges)} total nudges
Win Patterns: {len(win_patterns)} patterns identified
Loss Patterns: {len(loss_patterns)} patterns identified
"""

        # Create prompt
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a sales operations executive assistant. Your job is to create concise, actionable executive summaries of sales enablement reports.

Focus on:
1. Key metrics and their business implications
2. Critical issues requiring immediate attention
3. Top opportunities to pursue
4. Strategic recommendations

Keep it brief (3-4 paragraphs max), professional, and action-oriented."""),
            ("human", """Create an executive summary based on this sales enablement data:

{context}

Generate a concise summary that highlights:
- Overall pipeline health and key metrics
- Critical issues (stalled deals, at-risk deals, rep performance concerns)
- Top opportunities (high-priority leads, win patterns)
- Strategic recommendations

Format as clear, professional prose suitable for executive review.""")
        ])

        # Initialize LLM
        llm = ChatOpenAI(
            model=config.llm_model,
            temperature=config.temperature
        )

        # Generate summary
        chain = prompt | llm
        response = chain.invoke({"context": context})

        return response.content.strip()

    except Exception as e:
        # Graceful fallback - return None if LLM fails
        print(f"⚠️ LLM summary generation failed: {e}")
        return None



In [ ]:
def llm_summary_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    LLM Summary Node: Generate executive summary using LLM.

    Creates a concise, executive-friendly summary of the sales enablement insights.
    Falls back gracefully if LLM is unavailable.
    """
    errors = state.get("errors", [])
    config = SalesEnablementOrchestratorConfig()

    try:
        # Generate LLM summary
        summary = generate_executive_summary(state, config)

        if summary:
            # Save summary to separate file
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            summary_file_path = save_report(
                report_content=summary,
                report_id=timestamp,
                reports_dir=config.reports_dir,
                prefix="executive_summary"
            )

            return {
                "executive_summary": summary,
                "summary_file_path": summary_file_path,
                "errors": errors
            }
        else:
            # LLM failed or disabled - that's okay, we still have the full report
            return {
                "executive_summary": None,
                "summary_file_path": None,
                "errors": errors
            }
    except Exception as e:
        # Graceful fallback - don't fail the workflow if LLM fails
        return {
            "executive_summary": None,
            "summary_file_path": None,
            "errors": errors + [f"llm_summary_node: {str(e)} (non-critical)"]
        }


# Test Suite for Phase 7: Orchestrator Workflow"

In [ ]:
"""Test Suite for Phase 7: Orchestrator Workflow"""

from agents.sales_enablement.orchestrator import (
    create_sales_enablement_orchestrator,
    run_sales_enablement_orchestrator
)
from config import SalesEnablementOrchestratorState


def test_create_orchestrator():
    """Test orchestrator creation"""
    print("Testing create_orchestrator...")

    orchestrator = create_sales_enablement_orchestrator()

    assert orchestrator is not None, "should create orchestrator"

    print(f"✅ create_orchestrator test passed!")
    print(f"   Orchestrator created successfully")
    print()


def test_run_orchestrator_full():
    """Test running the full orchestrator workflow"""
    print("Testing run_orchestrator_full...")

    final_state = run_sales_enablement_orchestrator()

    # Check that all expected outputs are present
    assert "goal" in final_state, "should have goal"
    assert "plan" in final_state, "should have plan"
    assert "leads" in final_state, "should have leads"
    assert "prioritized_leads" in final_state, "should have prioritized_leads"
    assert "customer_needs_analysis" in final_state, "should have customer_needs_analysis"
    assert "outreach_recommendations" in final_state, "should have outreach_recommendations"
    assert "follow_up_actions" in final_state, "should have follow_up_actions"
    assert "rep_nudges" in final_state, "should have rep_nudges"
    assert "deal_insights" in final_state, "should have deal_insights"
    assert "win_patterns" in final_state, "should have win_patterns"
    assert "pipeline_summary" in final_state, "should have pipeline_summary"
    assert "enablement_report" in final_state, "should have enablement_report"
    assert "report_file_path" in final_state, "should have report_file_path"
    assert "executive_summary" in final_state, "should have executive_summary field"

    # Check for errors
    errors = final_state.get("errors", [])
    assert len(errors) == 0, f"should have no errors, but got: {errors}"

    print(f"✅ run_orchestrator_full test passed!")
    print(f"   Report: {final_state.get('report_file_path', 'N/A')}")
    if final_state.get('summary_file_path'):
        print(f"   Summary: {final_state.get('summary_file_path', 'N/A')}")
    print(f"   Top leads: {len(final_state.get('top_priority_leads', []))}")
    print(f"   Nudges: {len(final_state.get('rep_nudges', []))}")
    print(f"   Pipeline: {final_state.get('pipeline_summary', {}).get('active_deals', 0)} active deals")
    print()


def test_run_orchestrator_with_lead_id():
    """Test running orchestrator with specific lead_id"""
    print("Testing run_orchestrator_with_lead_id...")

    final_state = run_sales_enablement_orchestrator(lead_id="L-001")

    assert final_state.get("lead_id") == "L-001", "should have lead_id in state"
    assert "prioritized_leads" in final_state, "should have prioritized_leads"

    print(f"✅ run_orchestrator_with_lead_id test passed!")
    print(f"   Lead ID: {final_state.get('lead_id')}")
    print()


def test_run_orchestrator_with_focus_area():
    """Test running orchestrator with specific focus_area"""
    print("Testing run_orchestrator_with_focus_area...")

    final_state = run_sales_enablement_orchestrator(focus_area="lead_prioritization")

    assert final_state.get("focus_area") == "lead_prioritization", "should have focus_area in state"
    assert "goal" in final_state, "should have goal"

    print(f"✅ run_orchestrator_with_focus_area test passed!")
    print(f"   Focus Area: {final_state.get('focus_area')}")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 7: Orchestrator Workflow - Test Suite")
    print("=" * 60)
    print()

    test_create_orchestrator()
    test_run_orchestrator_full()
    test_run_orchestrator_with_lead_id()
    test_run_orchestrator_with_focus_area()

    print("=" * 60)
    print("✅ All Phase 7 tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_007_TEMPLATE copy % python run_sales_enablement.py
============================================================
Sales Enablement Orchestrator
============================================================

🚀 Starting Sales Enablement Orchestrator...
   Lead ID: All leads
   Rep ID: All reps
   Focus Area: All areas

============================================================
✅ Orchestrator Execution Complete
============================================================

📄 Full Report saved to: output/sales_enablement_reports/sales_enablement_20251229_175300_20251229_175300.md
📋 Executive Summary saved to: output/sales_enablement_reports/executive_summary_20251229_175306_20251229_175306.md
📊 Pipeline: 11 active deals, $1,204,000 value
🎯 Top Priority Leads: 10
🔔 Rep Nudges: 24
⏱️  Duration: 6.78 seconds



**Executive Summary: Sales Enablement Report**

The current pipeline health reflects a total of 15 deals, with 11 active and a cumulative pipeline value of \$1,204,000. The weighted pipeline value stands at \$479,300, indicating a win rate of 50%. However, we are facing critical issues with 2 stalled deals and 5 at-risk deals that require immediate attention to avoid potential revenue loss.

Rep performance varies significantly, with Alex Morgan leading with 3 deals and an impressive pipeline of $515,000, achieving 81% of quota. In contrast, Jordan Lee is underperforming with only 4 deals totaling \$152,000 and only 65.3% of quota, highlighting a need for targeted coaching. Priya Shah is performing well with 2 deals at \$402,000 and 95% of quota. Addressing the performance gap for Jordan is essential to enhance overall team productivity.

There are 10 top-priority leads identified, which present significant opportunities for revenue growth. Additionally, we have recognized 4 win patterns that can be leveraged to replicate success in future deals, as well as 4 loss patterns that should be analyzed to mitigate risks in upcoming negotiations.

To optimize pipeline performance, it is recommended to implement immediate coaching for Jordan Lee to improve his sales techniques and close rates. Additionally, focus on converting the identified top-priority leads while addressing the stalled and at-risk deals through strategic follow-ups and resource allocation. Leveraging the identified win patterns will also be crucial in enhancing our overall sales strategy and increasing the win rate moving forward.

Both reports look good. Here's what stands out:

## Full report — comprehensive and well-structured

Strengths:
- Pipeline metrics are accurate ($1,204,000 total, $479,300 weighted)
- Rep performance breakdown with clear indicators (✓ for good, ⚠️ for needs coaching)
- Top 10 priority leads ranked by score
- Actionable insights: 7 overdue follow-ups, 2 stalled deals, 5 at-risk deals
- Historical patterns: win/loss patterns with recommendations
- Clear sections for quick scanning

## Executive summary — concise and actionable

Strengths:
- 4 focused paragraphs
- Highlights critical issues upfront (stalled/at-risk deals)
- Rep performance with specific recommendations (Jordan Lee needs coaching)
- Strategic opportunities (10 priority leads, win patterns)
- Actionable recommendations at the end
- Professional tone suitable for executives

## What's working well

1. Separation of concerns:
   - Full report = detailed analysis for operations
   - Executive summary = quick insights for leadership

2. LLM quality:
   - Distills key points effectively
   - Maintains accuracy (numbers match the full report)
   - Provides actionable recommendations

3. Structure:
   - Both files are easy to navigate
   - Clear formatting and hierarchy
   - Professional presentation

## Minor observations

- The executive summary could include a brief mention of the 24 rep nudges generated (it mentions the leads and patterns but not the nudges)
- The full report shows "Average Days to Close: 0.0 days" — this might be due to data limitations, but it's not critical

Overall, the dual-report approach is working well. The full report provides depth, and the executive summary gives leadership a quick, actionable overview. The LLM summary effectively distills the key insights without losing important context.

# Sales Enablement Report

**Generated:** 2025-12-29 17:53:00  
**Objective:** Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights

---

## Executive Summary

### Pipeline Health

- **Total Deals:** 15
- **Active Deals:** 11
- **Won Deals:** 2
- **Lost Deals:** 2
- **Total Pipeline Value:** \$1,204,000
- **Weighted Pipeline Value:** \$479,300
- **Average Deal Size:** \$107,667
- **Average Days to Close:** 0.0 days
- **Win Rate:** 50.0%
- **Stalled Deals:** 2
- **At-Risk Deals:** 5

### Rep Performance

✓ **Alex Morgan** (SR-01)
   - Active Deals: 3
   - Pipeline Value: $515,000
   - Close Rate: 100.0%
   - Quota Achievement: 81.0%
   - Nudges: 2

⚠️ **Jordan Lee** (SR-02)
   - Active Deals: 4
   - Pipeline Value: $152,000
   - Close Rate: 0.0%
   - Quota Achievement: 65.3%
   - Nudges: 0

✓ **Priya Shah** (SR-03)
   - Active Deals: 2
   - Pipeline Value: $402,000
   - Close Rate: 50.0%
   - Quota Achievement: 95.0%
   - Nudges: 3

⚠️ **Miguel Alvarez** (SR-04)
   - Active Deals: 2
   - Pipeline Value: $135,000
   - Close Rate: 0.0%
   - Quota Achievement: 54.0%
   - Nudges: 2

---

## Top Priority Leads

1. **Orion Aerospace** (L-020) - Score: 86.6
2. **Apex Manufacturing** (L-003) - Score: 85.5
3. **Atlas Freight** (L-011) - Score: 83.0
4. **NovaEnergy Solutions** (L-006) - Score: 81.8
5. **OmniPharma** (L-015) - Score: 77.5
6. **Northstar Logistics** (L-001) - Score: 74.5
7. **Vertex Consulting** (L-014) - Score: 71.8
8. **Skyline Retail Group** (L-005) - Score: 70.2
9. **Horizon AgriTech** (L-018) - Score: 69.3
10. **ClearWave Health** (L-002) - Score: 62.6

---

## Customer Needs Analysis

### Lead L-001

- **Pain Points:** manual reporting, forecast inaccuracy
- **Buying Signals:** positive engagement
- **Product Fit Score:** 0.00

### Lead L-002

- **Pain Points:** data silos, slow reporting
- **Product Fit Score:** 0.00

### Lead L-003

- **Pain Points:** cost overruns, margin pressure
- **Buying Signals:** proposal requested, positive engagement, pricing discussed
- **Product Fit Score:** 0.00

### Lead L-004

- **Pain Points:** budget forecasting, grant reporting
- **Product Fit Score:** 0.00

### Lead L-005

- **Pain Points:** inventory forecasting, seasonality
- **Product Fit Score:** 0.00

---

## Outreach Recommendations

- **L-020** → SR-04 via email (N/A)
- **L-003** → SR-01 via email (N/A)
- **L-011** → SR-01 via email (N/A)
- **L-006** → SR-01 via email (N/A)
- **L-015** → SR-03 via call (N/A)

---

## Follow-up Actions

### ⚠️ Overdue (7)

- L-001 → SR-01: schedule_call
- L-003 → SR-01: send_proposal
- L-005 → SR-03: send_pricing
- L-008 → SR-04: schedule_call
- L-009 → SR-04: send_proposal

---

## Rep Nudges

- **Follow Up Due:** 7
- **Stalled Deal:** 2
- **High Priority Lead:** 10
- **Deal At Risk:** 5

### Sample Nudges

- **SR-01:** ⚠️ Follow-up with Northstar Logistics (L-001) is overdue. You promised to schedule demo 35 days ago....
- **SR-01:** ⚠️ Follow-up with Apex Manufacturing (L-003) is overdue. You promised to review proposal internally ...
- **SR-03:** ⚠️ Follow-up with Skyline Retail Group (L-005) is overdue. You promised to send pricing comparison 3...
- **SR-04:** ⚠️ Follow-up with Helix Biotech (L-008) is overdue. You promised to schedule follow-up 40 days ago....
- **SR-04:** ⚠️ Follow-up with Ironclad Construction (L-009) is overdue. You promised to review proposal 43 days ...

---

## Deal Insights

### Stalled Deals (2)

- D-004 (L-005): 21 days in current stage
- D-008 (L-009): 22 days in current stage

### At-Risk Deals (5)

- D-002 (L-002): pricing sensitivity
- D-004 (L-005): discount pressure, negative sentiment in recent interactions
- D-007 (L-008): budget uncertainty
- D-008 (L-009): timeline risk
- D-014 (L-015): legal review

---

## Historical Insights

### Win Patterns

- **Interaction Frequency:** Won deals had 1.0 interactions on average (range: 1-1) (100% frequency)
  → Recommendation: Aim for 1+ interactions for similar deals
- **Positive Sentiment:** 0/2 won deals had 2+ positive interactions (0% frequency)
  → Recommendation: Focus on building positive engagement early in the sales cycle
- **Pricing Discussion:** 0/2 won deals discussed pricing (0% frequency)
  → Recommendation: Introduce pricing discussion early for qualified leads

### Loss Patterns

- **Negative Sentiment:** 1/2 lost deals had negative sentiment (50% frequency)
  → Recommendation: Address negative sentiment immediately when detected
- **Risk Flags:** Most common risk flag in lost deals: 'pricing loss' (1 occurrences) (50% frequency)
  → Recommendation: Monitor and address 'pricing loss' risk flags proactively
- **Competition:** Most common competitor in lost deals: 'VendorX' (1 occurrences) (50% frequency)
  → Recommendation: Develop competitive differentiation strategy against 'VendorX'

---

*Report generated by Sales Enablement Orchestrator Agent*
